In [1]:
import numpy as np
from sklearn.datasets import make_blobs

In [22]:
centers = [[-5, 4]]
X, y1 = make_blobs(n_samples=20, centers=centers, random_state=20)
transformation = [[0.4, -0.1], [0.4, 1]]
X1 = np.dot(X, transformation)

centers = [[2, 3]]
X, y2 = make_blobs(n_samples=20, centers=centers, random_state=20)
transformation = [[0.2, -0.4], [0.8, 0.2]]
X2 = np.dot(X, transformation)
X = np.concatenate([X1, X2])
y = np.concatenate([y1, y2+1])

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [25]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

In [26]:
class BayesianLogisticRegression:
    def __init__(self, input_dim, alpha=1.0, iters=1000, lr=0.001):
        self.w = nn.Parameter(torch.randn(input_dim))
        self.iters = iters
        self.lr = lr
        self.alpha = alpha
    
    def _zero_grad(self):
        if self.w.grad is not None:
            self.w.grad.data.zero_()
    
    def _update_weights(self):
        self.w.data -= self.lr * self.w.grad.data
        
    def fit(self, X, y):
        for i in range(self.iters):
            pred = torch.sigmoid(torch.matmul(X, self.w))
            loss = F.binary_cross_entropy(pred, y) + 0.5 * self.alpha * torch.dot(self.w, self.w)
            self._zero_grad()
            loss.backward()
            self._update_weights()
        
        with torch.no_grad():
            p = torch.sigmoid(torch.matmul(X, self.w))
            self.w_precision = torch.matmul(X.T * p * (1 - p), X)
            self.w_precision[np.diag_indices_from(self.w_precision)] += self.alpha
    
    def predict(self, X):
        with torch.no_grad():
            mu = torch.matmul(X, self.w)
            var = (torch.linalg.solve(self.w_precision, X.T).T * X).sum(dim=1)
            return torch.sigmoid(mu / torch.sqrt(1 + torch.pi * var / 8))

In [27]:
clf = BayesianLogisticRegression(X.shape[1], 1, lr=0.01)
clf.fit(X, y)

In [29]:
pred = clf.predict(X)